In [ ]:
import numpy as np
from qibo import gates, hamiltonians, models
from scipy.optimize import minimize
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import argparse

In [ ]:
def hamiltonian(nqubits):
        m0 = (1/nqubits)*hamiltonians.Z(nqubits, numpy=True).matrix
        ham = hamiltonians.Hamiltonian(nqubits, m0)
        return ham
    
def generator(params, x):
    circuit = models.Circuit(nqubits)
    circuit.add(gates.RY(0, params[0]*x + params[1]))
    circuit.add(gates.RY(1, params[2]*x + params[3]))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.RY(0, params[4]*x + params[5]))
    circuit.add(gates.RY(1, params[6]*x + params[7]))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.RY(0, params[8]*x + params[9]))
    circuit.add(gates.RY(1, params[10]*x + params[11]))
    return circuit
    
def cost_generator(params, opt_params_disc, samples, bins, count):
    cost = 0
    #run generator
    shots_fake = sample_generator(params, samples, bins)
    #run discriminator
    for i in range(len(mean_bins)):
        state = discriminator(opt_params_disc, mean_bins[i]).execute()
        probs = (1 - encoder.expectation(state).numpy().real)/2
        cost -= shots_fake[i]*probs
                
    if count[0] % 1 == 0:
        print(count[0], cost)
    count[0] += 1      
        
    return cost

def discriminator(params, x):
    circuit = models.Circuit(nqubits)
    circuit.add(gates.RY(0, params[0]*x + params[1]))
    circuit.add(gates.RY(1, params[2]*x + params[3]))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.RY(0, params[4]*x + params[5]))
    circuit.add(gates.RY(1, params[6]*x + params[7]))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.RY(0, params[8]*x + params[9]))
    circuit.add(gates.RY(1, params[10]*x + params[11]))
    circuit.add(gates.CZ(0, 1))
    circuit.add(gates.RY(0, params[12]*x + params[13]))
    circuit.add(gates.RY(1, params[14]*x + params[15]))
    return circuit
        
def cost_discriminator(params, opt_params_gen, shots_real, shots_fake, count):
    cost = 0
    for i in range(len(mean_bins)):
        #run real and fake data
        state = discriminator(params, mean_bins[i]).execute()
        probs = (1 - encoder.expectation(state).numpy().real)/2
        cost += (shots_fake[i]-shots_real[i])*probs
        
    if count[0] % 50 == 0:
        print(count[0], cost)
    count[0] += 1
        
    return cost

def sample_gaussian(mu, sigma, samples, bins):
    s = np.random.normal(mu, sigma, samples)
    count, bins = np.histogram(s, bins)
    return count

def sample_generator(params, samples, bins):
    fake_gaussian_sampling = []
    for i in range(samples):
        x = np.random.uniform(-1, 1)
        fake_sample = encoder.expectation(generator(params, x).execute()).numpy().real
        fake_gaussian_sampling.append(fake_sample)
    count, bins = np.histogram(fake_gaussian_sampling, bins)
    return count

In [ ]:
nqubits = 2
encoder = hamiltonian(nqubits)
nparams_gen = 12
nparams_disc = 16
initial_params_disc = np.random.uniform(0, 2*np.pi, nparams_disc)
initial_params_gen = np.random.uniform(0, 2*np.pi, nparams_gen)

In [ ]:
samples = 2000
bins = np.linspace(-1.0, 1.0, 21)
mean_bins = np.linspace(-0.95, 0.95, 20)
count_real = sample_gaussian(0, 0.25, samples, bins)
print(count_real)

In [ ]:
count_fake = sample_generator(initial_params_gen, samples, bins)
print(count_fake)

In [ ]:
count = [0]
print('\n Discriminator results: ')
result_disc = minimize(lambda p: cost_discriminator(p, initial_params_gen, count_real, count_fake, count), initial_params_disc,
                          method='Powell', options={'maxiter': 1e3, 'maxfev': 1e3})
print('Final parameters: ', result_disc.x)
print('Final cost function: ', result_disc.fun)

count = [0]
print('\n Generator results: ')
result_gen = minimize(lambda p: cost_generator(p, result_disc.x, samples, bins, count), initial_params_gen,
                          method='Powell', options={'maxiter': 30, 'maxfev': 30})
print('Final parameters: ', result_gen.x)
print('Final cost function: ', result_gen.fun)

In [ ]:
count_fake = sample_generator(result_gen.x, samples, bins)
print(count_fake)

In [ ]:
for i in range(5):
    count = [0]
    print('\n Discriminator results: ')
    result_disc = minimize(lambda p: cost_discriminator(p, result_gen.x, count_real, count_fake, count), result_disc.x,
                              method='Powell', options={'maxiter': 1e3, 'maxfev': 1e3})
    print('Final parameters: ', result_disc.x)
    print('Final cost function: ', result_disc.fun)
    
    count = [0]
    print('\n Generator results: ')
    result_gen = minimize(lambda p: cost_generator(p, result_disc.x, samples, bins, count), result_gen.x,
                              method='Powell', options={'maxiter': 30, 'maxfev': 30})
    print('Final parameters: ', result_gen.x)
    print('Final cost function: ', result_gen.fun)
    
    count_fake = sample_generator(result_gen.x, samples, bins)
    print(count_fake)